# ***Distribución BAÑOS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre or 'df_junio_2023_queretaro' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro']

In [10]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria','Banos_Total':'baño_total','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [11]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Total de baños

In [12]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)
        df['baño_total'] = df['baño_total'].clip(upper=8) # Limitar los valores de 'baño_total' a un máximo de 8
        #df['baño_total'] = pd.cut(df['baño_total'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

Nombre del DataFrame: df_alfa_abril_2024_queretaro
baño_total  0.0   1.0  1.5  2.0  2.5  3.0  3.5  4.0  4.5  5.0  6.0  7.0  8.0
segmento                                                                    
A            13   448    0  335    0  145    0   60    0   13    0    0    0
B            15   503    0  466    2  129    0   11    0    3    0    0    0
C            12   330    3  168    1   57    0    2    0    1    0    0    1
D            10   238    4  159    0   38    0    1    0    0    1    0    0
E             0   275    5  129    2   78    5   25    2   16   18    8   13
L             0   207    0   57    0   34    0   72    0   45   18    7    4
S            51  2151    0  596    0  672    1  688    0  362   47    1    7
Nombre del DataFrame: df_alfa_agosto_2024_queretaro
baño_total  1.0  1.5  2.0  2.5  3.0   3.5  4.0  4.5  5.0  5.5  6.0  6.5  7.0  \
segmento                                                                       
A            17   15  371  348  209   192   

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10376\705114878.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10376\705114878.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10376\705114878.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [13]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
baño_total    0.0     0.9     1.0    1.5     2.0    2.5     3.0     3.5  \
segmento                                                                  
A            38.0  1615.0  1490.0   16.0  1379.0  349.0   662.0   192.0   
B            44.0  2061.0  1653.0   15.0  1887.0  441.0   466.0   102.0   
C            31.0  1158.0  1004.0   42.0   736.0  315.0   204.0    48.0   
D            27.0   352.0   778.0  133.0   753.0  109.0   119.0    20.0   
E             5.0    27.0  1004.0  207.0   551.0  132.0   347.0    94.0   
L             8.0     NaN   657.0    0.0   215.0    4.0   128.0    57.0   
S           137.0  2614.0  6468.0   22.0  2232.0  511.0  2445.0  1020.0   

baño_total     4.0    4.5     5.0    5.5    6.0   6.5   7.0  7.5   8.0  
segmento                                                                
A            267.0   31.0    47.0    1.0    1.0   0.0   3.0  0.0   6.0  
B             48.0    7.0     8.0    0.0    0.0   0.0   0.0  0.0   1.0  
C  

In [15]:
import os
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    for df in dataframes_list:
        df.replace(0.9, 1.0, inplace=True)
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    guardar_grafico_como_html(fig, 'g_bar_baños', carpeta='assets/graficas') # Para guardar la gráfica como archivo HTML
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Número de baños",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Segmento",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )    
    
    fig.show()    
    return fig

# Guardar la gráfica como archivo HTML en la carpeta especificada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
baño_total    0.0     1.0    1.5     2.0    2.5     3.0     3.5     4.0  \
segmento                                                                  
A            38.0  3105.0   16.0  1379.0  349.0   662.0   192.0   267.0   
B            44.0  3714.0   15.0  1887.0  441.0   466.0   102.0    48.0   
C            31.0  2162.0   42.0   736.0  315.0   204.0    48.0     8.0   
D            27.0  1130.0  133.0   753.0  109.0   119.0    20.0    12.0   
E             5.0  1031.0  207.0   551.0  132.0   347.0    94.0   151.0   
L             8.0   657.0    0.0   215.0    4.0   128.0    57.0   283.0   
S           137.0  9082.0   22.0  2232.0  511.0  2445.0  1020.0  2802.0   

baño_total    4.5     5.0    5.5    6.0   6.5   7.0  7.5   8.0  
segmento                                                        
A            31.0    47.0    1.0    1.0   0.0   3.0  0.0   6.0  
B             7.0     8.0    0.0    0.0   0.0   0.0  0.0   1.0  
C             5.0     2.0    0.0   

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10376\3627569921.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10376\3627569921.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10376\3627569921.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10376\3627569921.py:10: SettingWithCopyWarning:


A v

In [16]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
print("Tabla general total de registros:")
print(tabla_suma)


Tabla general total de registros:
baño_total
0.0      308.0
1.0    21705.0
1.5      463.0
2.0     8327.0
2.5     2027.0
3.0     4702.0
3.5     1608.0
4.0     3792.0
4.5      884.0
5.0     1851.0
5.5      197.0
6.0      344.0
6.5       46.0
7.0       88.0
7.5       14.0
8.0      116.0
dtype: float64


In [17]:
# Calcular el total de registros
total_registros = tabla_suma.sum()
# Calcular el porcentaje que representa cada número de baños único del total
porcentajes = (tabla_suma / total_registros) * 100

# Imprimir los porcentajes
print("Porcentaje de cada número de baños único en relación con el total de registros:", total_registros)
print(porcentajes)


Porcentaje de cada número de baños único en relación con el total de registros: 46472.0
baño_total
0.0     0.662765
1.0    46.705543
1.5     0.996299
2.0    17.918316
2.5     4.361766
3.0    10.117920
3.5     3.460148
4.0     8.159752
4.5     1.902221
5.0     3.983044
5.5     0.423911
6.0     0.740231
6.5     0.098984
7.0     0.189361
7.5     0.030126
8.0     0.249613
dtype: float64


In [18]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Lista de colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']

# Función para sumar tablas
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Función para calcular porcentajes
def calcular_porcentajes(tabla):
    # Calcular los porcentajes
    tabla_porcentajes = tabla / tabla.sum() * 100
    return tabla_porcentajes

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
porcentajes = calcular_porcentajes(tabla_suma) # Calcular porcentajes

# Crear el gráfico de pastel con colores especificados
figURE = go.Figure(data=[go.Pie(labels=porcentajes.index, values=porcentajes, sort=False, marker=dict(colors=colores))])
figURE.update_layout( title_x=0.5, title_font_size=20,margin=dict(l=10, r=10, t=10, b=10),  )
figURE.update_traces(rotation=45) # Ángulo de inicio de las porciones del gráfico

# Exportar gráfica como archivo HTML y mostrar la figura
def guardar_grafico_como_html(figURE, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(figURE, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(figURE, 'g_pie_baños', carpeta='assets/graficas')

# Mostrar la figura
figURE.show()